<p>
This script generates the following file: 
</p><p>
'locations_data' : contains names of all the districts, sub-districts and towns.
</p><p>
Manual data cleaning is done on 'locations_data' and that file is saved to 'locations_data_2'
</p><p>
finally strip and lower is done to generate <b>'locations_data_2.csv'</b>
<p>

In [1]:
# Importing the required libraries.
import pandas as pd
import xlrd
import os

In [2]:
# Getting the list of all the state wise excel files.
_,_,files = next(os.walk('./state xlsx/'))

# Making a single dataframe for districts, sub-districts, towns names.
locations_data = pd.read_excel('./state xlsx/'+files[0])
locations_data = (locations_data[['MDDS DTC','SUB-DISTRICT NAME']][1:]).drop_duplicates()

for file in files[1:]:
    data = pd.read_excel('./state xlsx/'+file)
    data = (data[['MDDS DTC','SUB-DISTRICT NAME']][1:]).drop_duplicates()
    locations_data = pd.concat([locations_data,data],axis=0)

# Resetting the dataframe index.
locations_data = locations_data.reset_index(drop=True)

In [3]:
# Getting the names of all the ambiguous places.
dict1 = dict(locations_data['SUB-DISTRICT NAME'].value_counts())
names = set(dict(i for i in dict1.items() if i[1]>1).keys())

# Collecting all the indexes of the ambiguous sub-districts/towns.
idxs = []
data = locations_data.values
for i in range(1,len(data)):
    if (data[i][1]=='Area not under any Sub-district') or (data[i][1] in names and data[i][0]==data[i-1][0]):
        idxs.append(i)

# Removing all the ambiguous names.
for i in idxs:
    locations_data = locations_data.drop(i)

# Some special cases.
locations_data.loc[(locations_data['MDDS DTC']==520) & (locations_data['SUB-DISTRICT NAME']=='Raigarh')] = [520,'Raigad']
locations_data.loc[(locations_data['MDDS DTC']==557) & (locations_data['SUB-DISTRICT NAME']=='Bijapur')] = [557,'Vijayapur']
locations_data = locations_data.drop(160)
locations_data = locations_data.drop(1293)

In [4]:
# Setting up and saving the final file.
locations_data.columns = ['District Id','Location Name']
locations_data.to_csv('locations_data.csv',index=None)

In [5]:
# strip and lower on location names.
data = pd.read_csv('locations_data_2.csv')
data = data[['District Id','Location Name']].values
for i in range(len(data)):
    data[i][1] = str(data[i][1]).strip().lower()
data = pd.DataFrame(data,columns=['District Id','Location Name'])
data.to_csv('locations_data_2.csv',index=None)